In [15]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil

In [34]:
base_dir=r"C:\Users\JOHAN\Downloads\dogs-vs-cats_base\train"
dest_dir=r"C:\Users\JOHAN\Downloads\dogs-vs-cats_dest"

train_path=os.path.join(dest_dir,"train")
val_path=os.path.join(dest_dir,"val")
test_path=os.path.join(dest_dir,"test")

In [17]:
#separamos los datos de entrenamiento (100 para ambos datos)
#para ello analizamos las rutas de acceso de perros

for i ,name in enumerate(["dog.{0}.jpg".format(i) for i in range(2000)]):
    path_origin=os.path.join(base_dir,name)
    path_dest=""
    if i<1000: #los primeros 1000 para el entrenamiento
        path_dest=os.path.join(dest_dir+r"\train\dogs",name)
    elif i<1500:
        path_dest=os.path.join(dest_dir+r"\val\dogs",name)
    else:
        path_dest=os.path.join(dest_dir+r"\test\dogs",name)
    
    shutil.copyfile(path_origin,path_dest)


In [18]:
#para los gatos
for i ,name in enumerate(["cat.{0}.jpg".format(i) for i in range(2000)]):
    path_origin=os.path.join(base_dir,name)
    path_dest=""
    if i<1000: #los primeros 1000 para el entrenamiento
        path_dest=os.path.join(dest_dir+r"\train\cats",name)
    elif i<1500:
        path_dest=os.path.join(dest_dir+r"\val\cats",name)
    else:
        path_dest=os.path.join(dest_dir+r"\test\cats",name)
    
    shutil.copyfile(path_origin,path_dest)

In [22]:
#ahora seguimos con el modelo
for name,path in [(arch.name,arch.path) for arch in os.scandir(dest_dir)]:
    print(name)
    for name_1,path_1 in [(arch.name,arch.path) for arch in os.scandir(path)]:
        print("{0} : {1}".format(name_1,len(os.listdir(path_1))))

test
cats : 500
dogs : 500
train
cats : 1000
dogs : 1000
val
cats : 500
dogs : 500


### seleccion del modelo

In [27]:
def build_model():
    model=tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(150,150,3)))
    model.add(tf.keras.layers.Conv2D(32,(3,3),activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Conv2D(64,(3,3),activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Conv2D(128,(3,3),activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Conv2D(128,(3,3),activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512,activation="relu"))
    model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
    
    #compilacion del modelo
    model.compile(loss="binary_crossentropy",
                 optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
                 metrics=["accuracy"]
                 )
    
    return model

In [28]:
model_conv=build_model()

In [29]:
model_conv.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 128)      

In [41]:
#ahora usaremos un generador de imagenes

from keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1.0/255)
validation_datagen=ImageDataGenerator(rescale=1.0/255)

In [42]:
#batch_size
batch_size=20
#train generator
train_generator=train_datagen.flow_from_directory(
        train_path,
        target_size=(150,150),
        batch_size=batch_size,
        class_mode="binary"
    )

#test generator
validation_generator=validation_datagen.flow_from_directory(
        val_path,
        target_size=(150,150),
        batch_size=batch_size,
        class_mode="binary"
    )

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [43]:
for data_batch,label_batch in validation_generator:
    print(data_batch.shape)
    print(label_batch.shape)
    
    break

(20, 150, 150, 3)
(20,)


In [49]:
model_conv=build_model()
history=model_conv.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples//batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples//batch_size,
        epochs=100
        )

Epoch 1/100
100/100 [==============================] - 60s 596ms/step - loss: 0.6890 - accuracy: 0.5410 - val_loss: 0.6835 - val_accuracy: 0.5420
Epoch 2/100
100/100 [==============================] - 49s 491ms/step - loss: 0.6563 - accuracy: 0.6210 - val_loss: 0.6470 - val_accuracy: 0.6170
Epoch 3/100
100/100 [==============================] - 51s 509ms/step - loss: 0.6208 - accuracy: 0.6650 - val_loss: 0.6340 - val_accuracy: 0.6250
Epoch 4/100
100/100 [==============================] - 50s 504ms/step - loss: 0.5737 - accuracy: 0.7050 - val_loss: 0.6147 - val_accuracy: 0.6560
Epoch 5/100
100/100 [==============================] - 51s 514ms/step - loss: 0.5357 - accuracy: 0.7305 - val_loss: 0.6781 - val_accuracy: 0.6250
Epoch 6/100
100/100 [==============================] - 50s 501ms/step - loss: 0.5139 - accuracy: 0.7415 - val_loss: 0.5718 - val_accuracy: 0.6970
Epoch 7/100
100/100 [==============================] - 50s 504ms/step - loss: 0.4802 - accuracy: 0.7765 - val_loss: 0.5673 -

KeyboardInterrupt: 

In [51]:
model_conv.evaluate(validation_generator)

50/50 [==============================] - 6s 122ms/step - loss: 0.5693 - accuracy: 0.7300


[0.5692743062973022, 0.7300000190734863]